In [2]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [11]:
CUDA = False
SEED = 1
BATCH_SIZE = 128
LOG_INTERVAL = 10
EPOCHS = 10

ZDIMS = 20

In [4]:
torch.manual_seed(SEED)

kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}

In [54]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs

)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)               




In [55]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(784, 400)
        
        self.relu = nn.ReLU()
        self.fc21 = nn.Linear(400, ZDIMS)
        self.fc22 = nn.Linear(400, ZDIMS)
        
        
        self.fc3 = nn.Linear(ZDIMS, 400)
        
        self.fc4 = nn.Linear(400, 784)
        self.sigmoid = nn.Sigmoid()
        
    
    
    def reparameterize(self, mu: Variable, logvar: Variable) -> Variable:
        
        if self.training:
            
            std = logvar.mul(.5).exp_()
            
            eps = Variable(std.data.new(std.size()).normal_())
            
            return eps.mul(std).add_(mu)
        else:
            return mu
    
    def encode(self, x: Variable) -> (Variable, Variable):
        
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
    def decode(self, z: Variable) -> Variable:
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x: Variable) -> (Variable, Variable, Variable):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

        

In [56]:
model = VAE()
if CUDA:
    model.cuda()


def loss_function(recon_x, x, mu, logvar) -> Variable:
    
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784))
    
    
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    KLD /= BATCH_SIZE * 784
    
    return BCE + KLD



In [57]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train(epoch):
    
    model.train()
    train_loss = 0
    
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        if CUDA:
            data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, logvar = model(data)
        
        loss = loss_function(recon_batch, data, mu, logvar)
        
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len (train_loader.dataset),
            100. * batch_idx / len(train_loader),
            loss.data[0] / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)
    ))

In [58]:
def test(epoch):
    
    model.eval()
    test_loss = 0
    
    for i, (data, _) in enumerate(test_loader):
        if CUDA:
            data = data.cuda()
        
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]
        
        if i == 0:
            
            n = min(data.size(0), 8)
            
            comparison = torch.cat([data[:n],
                                  recon_batch.view(BATCH_SIZE, 1, 28,28)[:n]])
            save_image(comparison.data.cpu(),
                      'results/reconstruction_' + str(epoch) + '.png', nrow=n)
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [59]:
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test(epoch)
    
    
    sample = Variable(torch.randn(64, ZDIMS))
    
    if CUDA:
        sample = sample.cuda()
    
    sample = model.decode(sample).cpu()
    
    
    save_image(sample.data.view(64, 1, 28, 28),
              'results/sample_' + str(epoch) + '.png')

IndexError: invalid index of a 0-dim tensor. Use tensor.item() to convert a 0-dim tensor to a Python number